In [64]:
import pandas as pd

df_operations = pd.read_excel(
    "k:/DOP/OED/METHOD&TOOLS/3 - PROJECTS/2 - ON GOING/2 - OE/2502 DIGITAL PROJECTS YEVGENIY/TRS/TRS_relations.xlsx",
    sheet_name='Operations')

In [65]:
df_operations = df_operations[['event', 'Category']]
mask = df_operations['Category'].notna()
df_operations_u = df_operations[mask].reset_index(drop=True)
df_operations_u

,event,Category
0,LG_PHASE_1,Planned_downtime
1,LG_PHASE_2,Planned_downtime
2,LG_PHASE_3,Planned_downtime
3,LG_PHASE_4_1,Planned_downtime
4,LG_PHASE_5_2,Planned_downtime
5,RP_BROKEN_CABLE,Unplanned_downtime_losses
6,RP_BROKEN_ELECTRICAL,Unplanned_downtime_losses
7,RP_BROKEN_GENERATOR,Unplanned_downtime_losses
8,RP_BROKEN_HYDRAULIC,Unplanned_downtime_losses
9,RP_BROKEN_PT_1200,Unplanned_downtime_losses


In [66]:
import pyodbc
print(pyodbc.drivers())

['SQL Server', 'SQL Server Native Client 11.0', 'ODBC Driver 17 for SQL Server']


In [67]:
driver = 'ODBC Driver 17 for SQL Server'
server = 'BKZTKDSDB41,21433'
database = 'PIFD'

# Строка подключения
conn_str = f'DRIVER={{{driver}}};SERVER={server};DATABASE={database};Trusted_Connection=yes;'

# SQL-запрос
query = """
SELECT 
    AFE.name COLLATE DATABASE_DEFAULT AS [equipment],
    AFET.name COLLATE DATABASE_DEFAULT AS [event],
    DATEADD(SECOND, (AVEvF.starttime / 10000000) - 63240134400 + 6 * 3600, '2005-01-01') AS starttime,
    IIF(AVEvF.endtime > 2638317188000000000, NULL, 
        DATEADD(SECOND, (AVEvF.endtime / 10000000) - 63240134400 + 6 * 3600, '2005-01-01')) AS endtime
FROM PIFD.dbo.AFEventFrame AS AVEvF
LEFT JOIN PIFD.dbo.AFElementTemplate AS AFET 
    ON AVEvF.fktemplateid = AFET.rid
LEFT JOIN PIFD.dbo.AFElement AS AFE 
    ON AVEvF.fkprimaryreferencedelement = AFE.rid
WHERE AFE.name COLLATE DATABASE_DEFAULT IN (
    SELECT DISTINCT equipment COLLATE DATABASE_DEFAULT 
    FROM PI_EXTRA.dbo.DR_REL_EQUIPMENT_EVENT
)
AND AFE.id IN (
    SELECT fkelementid 
    FROM PIFD.dbo.AFElementVersion 
    WHERE fktemplateid = (
        SELECT id 
        FROM PIFD.dbo.AFElementTemplate 
        WHERE name = 'RIG' 
        AND fkdatabaseid = (
            SELECT id FROM PIFD.dbo.AFDatabase WHERE name = 'DEM_DR'
        )
    )
)
"""

# Загрузка в DataFrame
try:
    with pyodbc.connect(conn_str) as conn:
        df_osi_operations = pd.read_sql(query, conn) # pyright: ignore[reportArgumentType]
        print(df_osi_operations.head())  # Проверка результата
except Exception as e:
    print("Ошибка при подключении или выполнении запроса:", e)

C:\Users\ykarabekov\AppData\Local\Temp\ipykernel_20264\3572333381.py:42: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_osi_operations = pd.read_sql(query, conn) # pyright: ignore[reportArgumentType]


       equipment         event           starttime             endtime
0  DBKAZ40 BL №1     RP_REPAIR 2021-08-20 12:00:00 2021-09-02 17:00:00
1  DBKAZ40 BL №1  CV_LONG_STOP 2020-04-27 08:00:00 2020-08-15 08:00:00
2  DBKAZ40 BL №1        WT_PPR 2020-02-18 08:00:00 2020-02-18 09:00:00
3  DBKAZ40 BL №1      WT_LUNCH 2020-02-18 12:00:00 2020-02-18 13:00:00
4  DBKAZ40 BL №1        WT_PPR 2020-02-18 20:00:00 2020-02-18 21:00:00


In [68]:
df_osi_operations['duration'] = df_osi_operations['endtime'] - df_osi_operations['starttime']

df_osi_operations['duration'] = df_osi_operations['duration'].dt.total_seconds() / 3600

df_osi_operations


,equipment,event,starttime,endtime,duration
0,DBKAZ40 BL №1,RP_REPAIR,2021-08-20 12:00:00,2021-09-02 17:00:00,317.0
1,DBKAZ40 BL №1,CV_LONG_STOP,2020-04-27 08:00:00,2020-08-15 08:00:00,2640.0
2,DBKAZ40 BL №1,WT_PPR,2020-02-18 08:00:00,2020-02-18 09:00:00,1.0
3,DBKAZ40 BL №1,WT_LUNCH,2020-02-18 12:00:00,2020-02-18 13:00:00,1.0
4,DBKAZ40 BL №1,WT_PPR,2020-02-18 20:00:00,2020-02-18 21:00:00,1.0
...,...,...,...,...,...
556782,ЯЯЯЯ-ФЕЙК,DR_CORE_DRILLING,2024-09-29 21:00:00,2024-09-29 23:30:00,2.5
556783,ЯЯЯЯ-ФЕЙК,DR_REAMING,2024-09-30 11:30:00,2024-09-30 13:30:00,2.0
556784,ЯЯЯЯ-ФЕЙК,DR_PUMP_REPLACEMENT,2024-09-30 14:00:00,2024-09-30 15:30:00,1.5
556785,ЯЯЯЯ-ФЕЙК,DR_WELL_FLUSHING,2024-10-01 12:30:00,2024-10-01 15:00:00,2.5


In [69]:
df_rigs = pd.read_excel(
    "k:/DOP/OED/METHOD&TOOLS/3 - PROJECTS/2 - ON GOING/2 - OE/2502 DIGITAL PROJECTS YEVGENIY/TRS/TRS_relations.xlsx",
    sheet_name='Rigs-Name')
df_rigs

,DrillCompany,Rig-AcQuire,Rig-osiDEM,Tipe of circulation
0,BurGeoProekt,PRAKLA_01-BGP,NaN,Direct
1,BurGeoProekt,PRAKLA_02-BGP,NaN,Direct
2,BurGeoProekt,PRAKLA_03-BGP,NaN,Direct
3,BurGeoProekt,PRAKLA_04-BGP,NaN,Direct
4,BurGeoProekt,ZIF1200_01-BGP,ZIF 1200 №1 Бургеопроект,Direct
...,...,...,...,...
157,VolkovGeology_#5,ZIF1200_36-E05,NaN,Direct
158,VolkovGeology_#5,ZIF1200_37-E05,NaN,Direct
159,VolkovGeology_#5,ZIF1200_38-E05,NaN,Direct
160,VolkovGeology_#5,ZIF1200_42-E05,NaN,Direct


In [70]:
df_circ_data = pd.read_excel(
    "k:/DOP/OED/METHOD&TOOLS/3 - PROJECTS/2 - ON GOING/2 - OE/2502 DIGITAL PROJECTS YEVGENIY/TRS/TRS_relations.xlsx",
    sheet_name='Standart avarage meters')
df_circ_data

,Circ,"Standard avarage drilling, m/h","time to well drill, h"
0,Direct,2.7,210
1,RC,5.4,87


In [71]:
df_osi_operations['endtime'].max()

Timestamp('2025-06-03 09:00:00')

In [72]:
database_a = 'acqkzdem1'
# SQL-запрос
query_2 = """
SELECT 
    HOLEID,
    DrillCompany,
    DrillRig,
    HolePurpose,
    HoleStatus,
    ENDDATE,
    [DEPTH]
FROM [acqkzdem1].[dbo].[AREVA_V_ARCGIS_DH_DEM]
"""

# Строка подключения
conn_str = f'DRIVER={{{driver}}};SERVER={server};DATABASE={database_a};Trusted_Connection=yes;'

# Загрузка данных в DataFrame
try:
    with pyodbc.connect(conn_str) as conn:
        df_meters = pd.read_sql(query_2, conn) # pyright: ignore[reportArgumentType]
        print(df_meters.head())  # Проверка
except Exception as e:
    print("Ошибка при подключении или выполнении запроса:", e)

C:\Users\ykarabekov\AppData\Local\Temp\ipykernel_20264\1744419047.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_meters = pd.read_sql(query_2, conn) # pyright: ignore[reportArgumentType]


        HOLEID  DrillCompany         DrillRig HolePurpose HoleStatus  \
0  MCI_N_227_2   Mechkolonna  ZIF1200_15-MK49       PIEZO   ACCEPTED   
1  MCI_N_228_1   Mechkolonna  ZIF1200_19-MK49       PIEZO   ACCEPTED   
2  MCI_N_229_1   Mechkolonna  ZIF1200_19-MK49       PIEZO   ACCEPTED   
3  MCI_N_231_1  BurGeoProekt   ZIF1200_04-BGP       PIEZO   ACCEPTED   
4  MCI_N_232_1  BurGeoProekt   ZIF1200_10-BGP       PIEZO   ACCEPTED   

     ENDDATE  DEPTH  
0 2024-10-13  424.0  
1 2025-01-01  420.0  
2 2024-12-10  422.0  
3 2024-07-27  428.0  
4 2024-07-22  428.0  


In [73]:
df_meters

,HOLEID,DrillCompany,DrillRig,HolePurpose,HoleStatus,ENDDATE,DEPTH
0,MCI_N_227_2,Mechkolonna,ZIF1200_15-MK49,PIEZO,ACCEPTED,2024-10-13,424.0
1,MCI_N_228_1,Mechkolonna,ZIF1200_19-MK49,PIEZO,ACCEPTED,2025-01-01,420.0
2,MCI_N_229_1,Mechkolonna,ZIF1200_19-MK49,PIEZO,ACCEPTED,2024-12-10,422.0
3,MCI_N_231_1,BurGeoProekt,ZIF1200_04-BGP,PIEZO,ACCEPTED,2024-07-27,428.0
4,MCI_N_232_1,BurGeoProekt,ZIF1200_10-BGP,PIEZO,ACCEPTED,2024-07-22,428.0
...,...,...,...,...,...,...,...
17900,TSU77_09_06_C_,BurServisSnab,PRAKLA_01-BSS,INJECT,ACCEPTED,2024-07-31,344.0
17901,TSU77_09_07_C_,Mechkolonna,ZIF1200_15-MK49,INJECT,ACCEPTED,2023-10-11,360.0
17902,TSU77_09_08_C_,Mechkolonna,ZIF1200_21-MK49,INJECT,ACCEPTED,2023-10-09,350.0
17903,TSU77_10_07_C_,BurGeoProekt,ZIF1200_03-BGP,PRODUCT,ACCEPTED,2024-07-08,359.0


df_operations     - Название операций <br>
df_operations_u   - Название операций уникальные значения <br>
df_osi_operations - Время операций <br>
df_rigs           - Название агрегатов для связи <br>
df_circ_data      - Стандартные коэфициенты в зависим от циркуляции (скорость бурения в час, и время на одну скважину - час) <br>
df_meters         - Пробуренные метры

## Идея 
1 связать df_osi_operations - df_rigs (Взять за основу Rig-AcQuire не пустые)
2 связать df_osi_operations - df_df_operations_u


In [74]:
mask = df_rigs['Rig-AcQuire'].notna()
df_rigs = df_rigs[mask].reset_index(drop=True)
df_rigs.info()
df_osi_operations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162 entries, 0 to 161
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   DrillCompany         162 non-null    object
 1   Rig-AcQuire          162 non-null    object
 2   Rig-osiDEM           74 non-null     object
 3   Tipe of circulation  162 non-null    object
dtypes: object(4)
memory usage: 5.2+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 556787 entries, 0 to 556786
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   equipment  556787 non-null  object        
 1   event      556787 non-null  object        
 2   starttime  556787 non-null  datetime64[ns]
 3   endtime    556779 non-null  datetime64[ns]
 4   duration   556779 non-null  float64       
dtypes: datetime64[ns](2), float64(1), object(2)
memory usage: 21.2+ MB


In [94]:
df_osi_operations['equipment'] = df_osi_operations['equipment'].str.strip().str.lower()
df_rigs['Rig-osiDEM'] = df_rigs['Rig-osiDEM'].str.strip().str.lower()

df_merged = df_osi_operations.merge(
    df_rigs,
    left_on='equipment',
    right_on='Rig-osiDEM',
    how='inner'  # или 'inner', если нужны только совпадающие строки
)

df_merged

,equipment,event,starttime,endtime,duration,DrillCompany,Rig-AcQuire,Rig-osiDEM,Tipe of circulation
0,dbkaz40 bl №1,RP_REPAIR,2021-08-20 12:00:00,2021-09-02 17:00:00,317.0,KATCO,DBKAZ40-1,dbkaz40 bl №1,RC
1,dbkaz40 bl №1,CV_LONG_STOP,2020-04-27 08:00:00,2020-08-15 08:00:00,2640.0,KATCO,DBKAZ40-1,dbkaz40 bl №1,RC
2,dbkaz40 bl №1,WT_PPR,2020-02-18 08:00:00,2020-02-18 09:00:00,1.0,KATCO,DBKAZ40-1,dbkaz40 bl №1,RC
3,dbkaz40 bl №1,WT_LUNCH,2020-02-18 12:00:00,2020-02-18 13:00:00,1.0,KATCO,DBKAZ40-1,dbkaz40 bl №1,RC
4,dbkaz40 bl №1,WT_PPR,2020-02-18 20:00:00,2020-02-18 21:00:00,1.0,KATCO,DBKAZ40-1,dbkaz40 bl №1,RC
...,...,...,...,...,...,...,...,...,...
505373,пбу змо-1500пс №6,WT_WAITING_FOR_LOGGING_5_2,2025-06-01 16:00:00,2025-06-01 19:00:00,3.0,KATCO,ZMO1500-6-KAT,пбу змо-1500пс №6,Direct
505374,пбу змо-1500пс №6,WT_WAITING_FOR_LOGGING_RESULT_5_2,2025-06-01 19:00:00,2025-06-01 21:00:00,2.0,KATCO,ZMO1500-6-KAT,пбу змо-1500пс №6,Direct
505375,пбу змо-1500пс №6,AS_MAST_DOWN,2025-06-01 22:00:00,2025-06-02 01:00:00,3.0,KATCO,ZMO1500-6-KAT,пбу змо-1500пс №6,Direct
505376,пбу змо-1500пс №6,AS_MAST_DOWN,2025-06-02 02:00:00,2025-06-02 09:00:00,7.0,KATCO,ZMO1500-6-KAT,пбу змо-1500пс №6,Direct


In [76]:
df_temp = df_merged.drop(columns=['equipment', 'Rig-osiDEM'])

In [90]:
df_operations_u

,event,Category
0,LG_PHASE_1,Planned_downtime
1,LG_PHASE_2,Planned_downtime
2,LG_PHASE_3,Planned_downtime
3,LG_PHASE_4_1,Planned_downtime
4,LG_PHASE_5_2,Planned_downtime
5,RP_BROKEN_CABLE,Unplanned_downtime_losses
6,RP_BROKEN_ELECTRICAL,Unplanned_downtime_losses
7,RP_BROKEN_GENERATOR,Unplanned_downtime_losses
8,RP_BROKEN_HYDRAULIC,Unplanned_downtime_losses
9,RP_BROKEN_PT_1200,Unplanned_downtime_losses


In [96]:
df_temp['event'] = df_temp['event'].str.strip().str.lower()
df_operations_u['event'] = df_operations_u['event'].str.strip().str.lower()

df_operations_total = df_temp.merge(df_operations_u, on='event', how='left')

df_operations_total['Category'] = df_operations_total['Category'].fillna('Speed_losess')

In [102]:
df_operations_copy = df_operations_total.copy()
df_operations_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505378 entries, 0 to 505377
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   event                505378 non-null  object        
 1   starttime            505378 non-null  datetime64[ns]
 2   endtime              505370 non-null  datetime64[ns]
 3   duration             505370 non-null  float64       
 4   DrillCompany         505378 non-null  object        
 5   Rig-AcQuire          505378 non-null  object        
 6   Tipe of circulation  505378 non-null  object        
 7   Category             505378 non-null  object        
dtypes: datetime64[ns](2), float64(1), object(5)
memory usage: 30.8+ MB


In [159]:
# Убедимся, что endtime — datetime (хотя уже подтверждено)
df_operations_total['endtime'] = pd.to_datetime(df_operations_total['endtime'])
df_operations_total = df_operations_total.dropna(subset=['endtime'])

# Добавим год и месяц
df_operations_total['year'] = df_operations_total['endtime'].dt.year.astype('int16')
df_operations_total['month'] = df_operations_total['endtime'].dt.month.astype('int8')


pivot_df = df_operations_total.pivot_table(
    index=['DrillCompany', 'Rig-AcQuire', 'year', 'month', 'Tipe of circulation'],
    columns='Category',
    values='duration',
    aggfunc='sum',
    fill_value=0  # если нужно подставить 0 вместо NaN
).reset_index()

pivot_df = pivot_df.sort_values(
    by=['DrillCompany', 'Rig-AcQuire', 'year', 'month'],
    ascending=[True, True, True, True]
).reset_index(drop=True)


pivot_df


Category,DrillCompany,Rig-AcQuire,year,month,Tipe of circulation,Planned_downtime,Speed_losess,Unplanned_downtime_losses
0,BurGeoProekt,ZIF1200_01-BGP,2020,2,Direct,0.0,0.0,276.0
1,BurGeoProekt,ZIF1200_01-BGP,2020,3,Direct,47.0,420.0,278.0
2,BurGeoProekt,ZIF1200_01-BGP,2020,4,Direct,17.0,182.0,28.0
3,BurGeoProekt,ZIF1200_01-BGP,2021,1,Direct,9.0,7015.0,93.0
4,BurGeoProekt,ZIF1200_01-BGP,2021,2,Direct,84.0,397.0,191.0
...,...,...,...,...,...,...,...,...
2804,TechnoService-Eng,ZIF1200_16-TSE,2023,3,Direct,89.0,509.0,90.0
2805,TechnoService-Eng,ZIF1200_16-TSE,2023,4,Direct,76.0,468.0,106.0
2806,TechnoService-Eng,ZIF1200_16-TSE,2023,5,Direct,76.0,486.0,119.0
2807,TechnoService-Eng,ZIF1200_16-TSE,2023,6,Direct,77.0,405.0,133.0


In [160]:
import calendar

def hours_in_month(row):
    days = calendar.monthrange(row['year'], row['month'])[1]
    return days * 24

pivot_df['h_in_month'] = pivot_df.apply(hours_in_month, axis=1)

pivot_df


Category,DrillCompany,Rig-AcQuire,year,month,Tipe of circulation,Planned_downtime,Speed_losess,Unplanned_downtime_losses,h_in_month
0,BurGeoProekt,ZIF1200_01-BGP,2020,2,Direct,0.0,0.0,276.0,696
1,BurGeoProekt,ZIF1200_01-BGP,2020,3,Direct,47.0,420.0,278.0,744
2,BurGeoProekt,ZIF1200_01-BGP,2020,4,Direct,17.0,182.0,28.0,720
3,BurGeoProekt,ZIF1200_01-BGP,2021,1,Direct,9.0,7015.0,93.0,744
4,BurGeoProekt,ZIF1200_01-BGP,2021,2,Direct,84.0,397.0,191.0,672
...,...,...,...,...,...,...,...,...,...
2804,TechnoService-Eng,ZIF1200_16-TSE,2023,3,Direct,89.0,509.0,90.0,744
2805,TechnoService-Eng,ZIF1200_16-TSE,2023,4,Direct,76.0,468.0,106.0,720
2806,TechnoService-Eng,ZIF1200_16-TSE,2023,5,Direct,76.0,486.0,119.0,744
2807,TechnoService-Eng,ZIF1200_16-TSE,2023,6,Direct,77.0,405.0,133.0,720


In [153]:
df_meters['year'] = df_meters['ENDDATE'].dt.year.astype('int16')
df_meters['month'] = df_meters['ENDDATE'].dt.month.astype('int8')

df_meters

,HOLEID,DrillCompany,DrillRig,HolePurpose,HoleStatus,ENDDATE,DEPTH,year,month
0,MCI_N_227_2,Mechkolonna,ZIF1200_15-MK49,PIEZO,ACCEPTED,2024-10-13,424.0,2024,10
1,MCI_N_228_1,Mechkolonna,ZIF1200_19-MK49,PIEZO,ACCEPTED,2025-01-01,420.0,2025,1
2,MCI_N_229_1,Mechkolonna,ZIF1200_19-MK49,PIEZO,ACCEPTED,2024-12-10,422.0,2024,12
3,MCI_N_231_1,BurGeoProekt,ZIF1200_04-BGP,PIEZO,ACCEPTED,2024-07-27,428.0,2024,7
4,MCI_N_232_1,BurGeoProekt,ZIF1200_10-BGP,PIEZO,ACCEPTED,2024-07-22,428.0,2024,7
...,...,...,...,...,...,...,...,...,...
17900,TSU77_09_06_C_,BurServisSnab,PRAKLA_01-BSS,INJECT,ACCEPTED,2024-07-31,344.0,2024,7
17901,TSU77_09_07_C_,Mechkolonna,ZIF1200_15-MK49,INJECT,ACCEPTED,2023-10-11,360.0,2023,10
17902,TSU77_09_08_C_,Mechkolonna,ZIF1200_21-MK49,INJECT,ACCEPTED,2023-10-09,350.0,2023,10
17903,TSU77_10_07_C_,BurGeoProekt,ZIF1200_03-BGP,PRODUCT,ACCEPTED,2024-07-08,359.0,2024,7


In [156]:
df_meters = df_meters.dropna(subset=['year', 'month', 'DEPTH'])

pivot_meters = df_meters.pivot_table(
    index=['DrillRig', 'year', 'month'],  # группируем по году, месяцу и статусу
    columns='HoleStatus',
    values='DEPTH',
    aggfunc='sum',
    fill_value=0  # если нужно заполнить NaN нулями
).reset_index()

pivot_meters = pivot_meters.sort_values(
    by=['DrillRig', 'year', 'month'],
    ascending=[True, True, True]
).reset_index(drop=True)

pivot_meters



HoleStatus,DrillRig,year,month,ACCEPTED,LIQUID,NOT PROFITABLE
0,DBKAZ40-1,2007,11,486.20,0.0,0.0
1,DBKAZ40-1,2007,12,478.00,471.6,0.0
2,DBKAZ40_1-KAT,2007,7,0.00,480.2,0.0
3,DBKAZ40_1-KAT,2007,8,1459.60,476.0,0.0
4,DBKAZ40_1-KAT,2007,9,1438.00,0.0,0.0
...,...,...,...,...,...,...
5852,ZMO1500-6-KAT,2025,1,1677.00,0.0,0.0
5853,ZMO1500-6-KAT,2025,2,1279.00,0.0,0.0
5854,ZMO1500-6-KAT,2025,3,1686.25,0.0,0.0
5855,ZMO1500-6-KAT,2025,4,1275.00,0.0,0.0


In [158]:
total_merged_df = pd.merge(
    pivot_df,
    pivot_meters,
    left_on=['Rig-AcQuire', 'year', 'month'],
    right_on=['DrillRig', 'year', 'month'],
    how='right'  # можно 'left', если хочешь все строки из pivot_df
)

total_merged_df

,DrillCompany,Rig-AcQuire,year,month,Tipe of circulation,Planned_downtime,Speed_losess,Unplanned_downtime_losses,h_in_month,DrillRig,ACCEPTED,LIQUID,NOT PROFITABLE
0,NaN,NaN,2007,11,NaN,NaN,NaN,NaN,NaN,DBKAZ40-1,486.20,0.0,0.0
1,NaN,NaN,2007,12,NaN,NaN,NaN,NaN,NaN,DBKAZ40-1,478.00,471.6,0.0
2,NaN,NaN,2007,7,NaN,NaN,NaN,NaN,NaN,DBKAZ40_1-KAT,0.00,480.2,0.0
3,NaN,NaN,2007,8,NaN,NaN,NaN,NaN,NaN,DBKAZ40_1-KAT,1459.60,476.0,0.0
4,NaN,NaN,2007,9,NaN,NaN,NaN,NaN,NaN,DBKAZ40_1-KAT,1438.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5852,KATCO,ZMO1500-6-KAT,2025,1,Direct,143.5,463.0,137.5,744.0,ZMO1500-6-KAT,1677.00,0.0,0.0
5853,KATCO,ZMO1500-6-KAT,2025,2,Direct,123.5,390.5,157.0,672.0,ZMO1500-6-KAT,1279.00,0.0,0.0
5854,KATCO,ZMO1500-6-KAT,2025,3,Direct,148.5,456.5,140.0,744.0,ZMO1500-6-KAT,1686.25,0.0,0.0
5855,KATCO,ZMO1500-6-KAT,2025,4,Direct,136.0,504.5,79.5,720.0,ZMO1500-6-KAT,1275.00,0.0,0.0
